In [ ]:
!pip install git+https://github.com/ArcetriAdaptiveOptics/OAO24.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from oao24.package_data import InfraredExampleData
from oao24 import image_processing

In [ ]:
background_image = InfraredExampleData.get_camera_dark_data()
cl_raw_image_cube = InfraredExampleData.get_close_loop_data_cube()
    
ao_image = image_processing.make_master_image(cl_raw_image_cube, background_image)